In [3]:
import os
import utils
import connectome_create
# viz_method = one of ['itkwidgets', 'vtk']
viz_method = 'vtk'

# import some of our favorite packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sea
import re
import warnings

%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
import nglui.statebuilder as ngstbld

# this is the EM specific package for querying the EM data
from caveclient import CAVEclient

from datetime import date
from meshparty import trimesh_io, trimesh_vtk
from meshparty import skeletonize, skeleton_io, skeleton
import cloudvolume

%load_ext autoreload
%autoreload 2

query timestamp: 2022-08-31 00:00:00
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from pathlib import Path
import json

with open(Path.home() / '.cloudvolume/secrets/'/'cave-secret.json') as f:
        tokens = json.load(f)
        
seg_source = 'graphene://https://cave.fanc-fly.com/segmentation/table/mar2021_prod'
cv = cloudvolume.CloudVolume(cloudpath=seg_source, use_https=True, secrets=tokens)

In [2]:
datastack_name = 'fanc_production_mar2021'

client = CAVEclient(datastack_name)

In [ ]:
# client = CAVEclient()

# # if not os.path.isfile(os.path.expanduser("~/.cloudvolume/secrets/cave-secret.json")):
# client.auth.get_new_token(open=True)

In [ ]:
# if you have not yet setup this computer, uncomment this below line
# paste the token from the website in, and run the line

# client.auth.save_token(token="c14cd7a3e18a1a697716a399afbf5778", overwrite=True)

# then comment or delete the line as you don't need to run it on this computer  again

In [ ]:
pre_to_mn_df = connectome_create.left_pre_to_mn_df(client)

# Query the annotation table 'motor_neuron_table_v7' 
'motor_neuron_table_v2' contains a list of all the right and left motor neurons in T1.

In [ ]:
client.materialize.get_tables()
t1_mns_df = client.materialize.query_table('motor_neuron_table_v7')
soma_table = client.materialize.query_table('soma_jan2022')
sensory_axons = client.materialize.query_table('sensory_axon_table')

t1_mns_df.head(70)


In [ ]:
sensory_axons

In [ ]:
sides = sorted([*{*side}])
nerves = sorted([*{*nerve}])
segments = sorted([*{*segment}])
fcns = sorted([*{*function}])

pre_to_mn_df = None

for sd in sides:
    mnmi_side = utils.multiindex_include(mn_index,[sd])
    for nrv in nerves: # ['Leg']:
        mnmi_side_nrv = utils.multiindex_include(mnmi_side,[nrv])
        segs = mnmi_side_nrv.get_level_values('segment').to_list()
        segs = sorted([*{*segs}])
        for sg in segs:
            mnmi_side_nrv_seg = utils.multiindex_include(mnmi_side_nrv,[sg])
            segfcns = mnmi_side_nrv_seg.get_level_values('function').to_list()
            segfcns = sorted([*{*segfcns}])
            for fcn in segfcns:
                mnmi_side_nrv_seg_fcn = utils.multiindex_include(mnmi_side_nrv_seg,[fcn])

                print('({}, {}, {}, {})'.format(sd,nrv,sg,fcn))

                # get the resulting segIDs
                mn_segIDs = mnmi_side_nrv_seg_fcn.get_level_values('segID').to_list()

                # Query the synapse table
                mn_inputs_df = client.materialize.synapse_query(post_ids = mn_segIDs) # Takes list
                mn_inputs_df['has_soma'] = mn_inputs_df.pre_pt_root_id.isin(soma_table.pt_root_id)
                mn_inputs_df['sensory'] = mn_inputs_df.pre_pt_root_id.isin(sensory_axons.pt_root_id)
                mn_inputs_df['sensory'] = mn_inputs_df.pre_pt_root_id.isin(sensory_axons.pt_root_id)
                print(mn_inputs_df.shape)

                mn_inputs_df = utils.group_and_count_inputs(mn_inputs_df,3)
                partner_df = utils.create_pre_post_df(mn_inputs_df,mnmi_side_nrv_seg_fcn)

                if pre_to_mn_df is None:
                    pre_to_mn_df = partner_df
                else:
                    # preserve the order of premotor neurons from most connected down
                    curidx = pre_to_mn_df.index.to_list()
                    newidx = partner_df.index.to_list()
                    for segid in newidx:
                        if segid not in curidx:
                            curidx.append(segid)
                    pre_to_mn_df = pd.concat([pre_to_mn_df,partner_df],axis=1,join='outer').reindex(index=curidx).fillna(value=0,downcast='infer')

mn_multidx = pre_to_mn_df.columns           
pre_to_mn_df

# Order according to segment (thorax, coxa, etc) and function (flex, extend, etc)

In [ ]:
# visualize matrix
mn_multidx = mn_multidx
mn_labels = mn_multidx.get_level_values('muscle').to_list()
mn_segment = mn_multidx.get_level_values('segment').to_list()
mn_fcn = mn_multidx.get_level_values('function').to_list()
mn_rank = mn_multidx.get_level_values('rank').to_list()
idx = [str(i) for i in range(len(mn_fcn))]
mn_rank = [str(i) for i in mn_rank]
lbls = [i + '_' + j + '_' + k + '_' + l + '_' + n for i, j, k, l, n in zip(idx, mn_segment, mn_fcn, mn_labels,mn_rank)]
lbls

In [ ]:
reorder_bysegfcn = utils.get_segment_fcn_order(side='')
np.array(lbls)[reorder_bysegfcn].tolist()

In [ ]:
# bristle_id = 648518346480454913
# pmn = pmnsegids[0:10]
# #Downloading Mesh
# pmn_cv_mesh = cv.mesh.get(pmn, remove_duplicate_vertices=True,use_byte_offsets=True)[pmn]
# mp_pmn = trimesh_io.Mesh(pmn_cv_mesh.vertices,pmn_cv_mesh.faces,pmn_cv_mesh.normals)
# print(pmn_cv_mesh.vertices.shape,pmn_cv_mesh.faces.shape,pmn_cv_mesh.normals.shape)

# max_dist = 1000
# dist_step=500
# ccs = scipy.sparse.csgraph.connected_components(mp_bristle.csgraph)
# ccs_u, cc_sizes = np.unique(ccs[1], return_counts=True)
# large_cc_ids = ccs_u[cc_sizes > 70]
# print(len(large_cc_ids))

# kdtrees = []
# vertex_ids = []
# for cc_id in large_cc_ids:
#     m = ccs[1] == cc_id
#     vertex_ids.append(np.where(m)[0])
#     v = mp_bristle.vertices[m]
#     kdtrees.append(spatial.cKDTree(v))

# add_edges = []
# for i_tree in range(len(large_cc_ids) - 1):
#     for j_tree in range(i_tree + 1, len(large_cc_ids)):
#         print("%d - %d      " % (i_tree, j_tree), end="\r")

#         if np.any(kdtrees[i_tree].query_ball_tree(kdtrees[j_tree], max_dist)):
#             for this_dist in range(dist_step, max_dist + dist_step, dist_step):

#                 pairs = kdtrees[i_tree].query_ball_tree(kdtrees[j_tree],
#                                                         this_dist)

#                 if np.any(pairs):
#                     for i_p, p in enumerate(pairs):
#                         if len(p) > 0:
#                             add_edges.extend([[vertex_ids[i_tree][i_p],
#                                                vertex_ids[j_tree][v]]
#                                               for v in p])
#                     break
# if len(add_edges) >0:
#     if large_cc_ids[1].size >0:

### Creating a Meshwork Object
#Depending your skeleton you may want to play around with the max_dist and other parameters. If you get an error about 
# 0 indices being merged, uncomment the code above and try that.
# mp_pmn.merge_large_components(max_dist=1000)
# in_comp = mesh_filters.filter_largest_component(mp_pmn)
# bristle_anchor = mp_bristle.apply_mask(in_comp)
# bristle_mw = meshwork.Meshwork(bristle_anchor, seg_id=bristle_id,voxel_resolution=[4.3,4.3,45])

# ## Skeletonizing
# ##I set the entry point as "soma_point" so that I can control the root, you can point any point you choose there or none
# #entry_pt = bristle_entry_points.entry_point_nm[bristle_entry_points['Description'] == bristle_id].tolist()[0]
# bristle_mw.skeletonize_mesh(entry_pt)

# Slice entire data frame to find particular segments

In [ ]:
All = slice(None)
pre_to_mn_df.shape

## Premotor neurons with SOMA: pre_wsoma_to_mn_df

In [ ]:
pre_wsoma_to_mn_df = pre_to_mn_df.loc[(All, True), :]
pre_wsoma_to_mn_df.shape

## Fragments with NO SOMA: pre_nosoma_to_mn_df

In [ ]:
pre_nosoma_to_mn_df = pre_to_mn_df.loc[(All, False), :]
pre_nosoma_to_mn_df.shape

## Left with soma: pre_soma_to_mn_L_df

In [ ]:
pre_wsoma_to_mn_L_df = pre_to_mn_df.loc[(All, True), ('L')]
pre_wsoma_to_mn_L_df.shape
numinputs = pre_wsoma_to_mn_L_df.sum(axis=1)
pre_wsoma_to_mn_L_df = pre_wsoma_to_mn_L_df.loc[numinputs>0,:]
pre_wsoma_to_mn_L_df.shape

## Right with soma: pre_soma_to_mn_R_df

In [ ]:
pre_wsoma_to_mn_R_df = pre_to_mn_df.loc[(All, True), ('R')]
pre_wsoma_to_mn_R_df.shape
numinputs = pre_wsoma_to_mn_R_df.sum(axis=1)
pre_wsoma_to_mn_R_df = pre_wsoma_to_mn_R_df.loc[numinputs>0,:]
pre_wsoma_to_mn_R_df.shape

## How do numbers of objects change over time?

In [ ]:
# Watch the number of objects with a soma increase
import datetime

datearr = [
    datetime.date(2022,4,20),
    datetime.date(2022,4,22),
    datetime.date(2022,4,25),
    datetime.date(2022,5,2)
]
NsegWsoma = [1770,1775,1782,1802]
NsegNoSoma = [np.nan,5646,5642,5599]

arrays = [
    datearr,
    NsegWsoma,
    NsegNoSoma,
        ]

fig = plt.figure(1, figsize=(12, 6))
ax1 = plt.subplot2grid((1,2),(0,0))

ax1.scatter(datearr, NsegWsoma)
plt.sca(ax1)
plt.ylabel('soma-attached objects')
plt.xlabel('date')
plt.title('Objects with somas')

ax2 = plt.subplot2grid((1,2),(0,1))
ax2.scatter(datearr, NsegNoSoma)
plt.ylabel('soma-attached objects')
plt.xlabel('date')
plt.title('Objects without somas')

plt.show()

## Premotors with somas that provide input to both right and left MNs

In [ ]:
numLinputs = np.array(pre_wsoma_to_mn_df.loc[(All, True), ('L')].sum(axis=1).tolist())
numRinputs = np.array(pre_wsoma_to_mn_df.loc[(All, True), ('R')].sum(axis=1).tolist())
bothinputs = (numLinputs >0) & (numRinputs >0)# numRinputs.ge(1)
pre_to_RL_mn_df = pre_wsoma_to_mn_df.loc[bothinputs,:]

# eliminate blank synapses
numinputs = np.array(pre_to_RL_mn_df.sum(axis=0).tolist())
pre_to_RL_mn_df = pre_to_RL_mn_df.loc[:,numinputs>0]

pre_to_RL_mn_df
today = date.today()
d1 = today.strftime("%Y%m%d")
pre_to_RL_mn_df.to_csv('./saved_dfs/preMNsTargetingRandL' + d1 + '.csv')
print(pre_to_RL_mn_df.shape)

### Compare the number of connections onto left and right neurons

In [ ]:
import matplotlib.text as text

numLinputs = np.array(pre_wsoma_to_mn_df.loc[(All, True), ('L')].sum(axis=1).tolist())
numRinputs = np.array(pre_wsoma_to_mn_df.loc[(All, True), ('R')].sum(axis=1).tolist())
bothinputs = (numLinputs >0) & (numRinputs >0)# numRinputs.ge(1)
pre_to_RL_mn_df = pre_wsoma_to_mn_df.loc[bothinputs,:]

numinputs = np.array(pre_to_RL_mn_df.sum(axis=0).tolist())
pre_to_RL_mn_df = pre_to_RL_mn_df.loc[:,numinputs>0]

leftinputs = pre_to_RL_mn_df.loc[All,('L')].sum(axis=1).to_numpy()
rightinputs = pre_to_RL_mn_df.loc[All,('R')].sum(axis=1).to_numpy()

lorder = np.flip(np.argsort(leftinputs))
li = leftinputs[lorder]
ri = rightinputs[lorder]
rlpms = pre_to_RL_mn_df.index.get_level_values('segID').to_numpy()
rlpms = rlpms[lorder]

In [ ]:
pre_to_RL_mn_df.iloc[lorder,:]

In [ ]:
labels = [str(i) for i in rlpms]
labels[0] = 'ascend.'
labels[1] = 'm to l'
labels[1] = 'm to l'
labels[4] = 'ascend.'

### Check out a few interesting RL neurons

In [ ]:
pmn_syn = client.materialize.synapse_query(pre_ids = rlpms[idx]) # Takes list
# client.materialize.synapse_query?
print(pmn_syn.shape)
pmn_syn_w_mn = pmn_syn.post_pt_root_id.isin(pre_to_RL_mn_df.columns.get_level_values('segID'))
pmn_syn = pmn_syn.loc[pmn_syn_w_mn,:]
pmn_syn

In [ ]:
# Create a neuroglancer link with a synapse layer
# Modified from Leila
import nglui.statebuilder as ngstbld

# variables to render in neuroglancer link
render_synapses = pmn_syn
render_neurons = [rlpms[idx]]

img_source = client.info.image_source()
img_layer = ngstbld.ImageLayerConfig(name='fanc_v4',
                             source=img_source,
                             )
seg_source = client.info.segmentation_source()

seg_layer = ngstbld.SegmentationLayerConfig(name = 'seg',
                                    source = seg_source,
                                    fixed_ids = render_neurons)
points = ngstbld.PointMapper(point_column='post_pt_position') ######################## change this to toggle rendering of pre- or post- synaptic points
anno_layer = ngstbld.AnnotationLayerConfig(name='annos',
                                   mapping_rules=points )

sb = ngstbld.StateBuilder([img_layer, seg_layer, anno_layer], resolution=[4.3,4.3,45])
sb.render_state(render_synapses, return_as='html')

In [ ]:
fig = plt.figure(1, figsize=(12, 9))
ax1 = plt.subplot2grid((1,1),(0,0))

plt.scatter(leftinputs, rightinputs)
plt.xlabel('inputs to L MNs')
plt.ylabel('inputs to R MNs')
plt.title('Presynaptic to L AND R MNs')

t = text.Text(li[4]+5, ri[4]+5, labels[4], ha='left', va='bottom',axes = ax1)
ax1.add_artist(t)

plt.show();

In [ ]:
pre_to_mn_dfto_numpy

# Cosine similarity comparison

## Compute cosine similarity, then cluster using agglomerative clustering

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html
# cosine similarity - K(X, Y) = <X, Y> / (||X||*||Y||)
sim_mat = cosine_similarity(pre_wsoma_to_mn_L_df.to_numpy().transpose())

# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None).fit(sim_mat)

plt.title("Hierarchical Clustering Dendrogram")
# plot the top three levels of the dendrogram
clustered_order = utils.plot_dendrogram(model, truncate_mode="level", p=12) # p truncate mode
clustered_order = np.array(clustered_order).astype(int) # convert strins into integers
print(clustered_order)

In [ ]:
reordered_wsoma_df = pre_wsoma_to_mn_L_df.iloc[:,clustered_order]
#reordered_mn_df = pre_to_mn_df.iloc[:,clustered_order]

In [ ]:
sim_mat = cosine_similarity(reordered_wsoma_df.to_numpy().transpose())
ro_multidx = reordered_wsoma_df.columns

# visualize matrix
mn_labels = ro_multidx.get_level_values('muscle').to_list()
mn_rank = ro_multidx.get_level_values('rank').to_list()
mn_ids = ro_multidx.get_level_values('segID').to_list()
idx = [str(i) for i in range(len(mn_ids))]
sid = [str(i) for i in mn_ids]
mn_rank = [str(i) for i in mn_rank]
lbls = [i + '_' + j + '_' + k + '_' + l for i, j, k, l in zip(mn_labels, mn_rank, sid, idx,)]

fig = plt.figure(1, figsize = [20,18])
ax = sea.heatmap(sim_mat, xticklabels=mn_ids, yticklabels=lbls)
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

## Order according to SEGMENT and FUNCTION

In [ ]:
reorder_idx_bysegfcn = utils.get_segment_fcn_order(side = 'left')
pre_wsoma_to_mn_L_df = pre_to_mn_df.loc[(All, True), ('L')]
numinputs = pre_wsoma_to_mn_L_df.sum(axis=1)
pre_wsoma_to_mn_L_df = pre_wsoma_to_mn_L_df.loc[numinputs>0,:]


fig2order_wsoma_df = pre_wsoma_to_mn_L_df.iloc[:,reorder_idx_bysegfcn]
#reordered_mn_df = pre_to_mn_df.iloc[:,clustered_order]
sim_mat = cosine_similarity(fig2order_wsoma_df.to_numpy().transpose())
ro_multidx = fig2order_wsoma_df.columns

# visualize matrix
mn_labels = ro_multidx.get_level_values('muscle').to_list()
mn_rank = ro_multidx.get_level_values('rank').to_list()
mn_ids = ro_multidx.get_level_values('segID').to_list()
idx = [str(i) for i in range(len(mn_ids))]
sid = [str(i) for i in mn_ids]
mn_rank = [str(i) for i in mn_rank]
lbls = [i + '_' + j + '_' + k + '_' + l for i, j, k, l in zip(mn_labels, mn_rank, sid, idx,)]

fig = plt.figure(1, figsize = [20,18])
ax = sea.heatmap(sim_mat, xticklabels=mn_ids, yticklabels=lbls)
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

# Cosine similarity of both sides

In [ ]:
# Total number of input sources WITH SOMA
pre_wsoma_to_mn_df.shape

In [ ]:
reorder_idx_bysegfcn = utils.get_segment_fcn_order()
pre_wsoma_to_mn_df = pre_to_mn_df.loc[(All, True), :]
fullfig2order_wsoma_df = pre_wsoma_to_mn_df.iloc[:,reorder_idx_bysegfcn]
#reordered_mn_df = pre_to_mn_df.iloc[:,clustered_order]
sim_mat = cosine_similarity(fullfig2order_wsoma_df.to_numpy().transpose())
ro_multidx = fullfig2order_wsoma_df.columns

# visualize matrix
mn_side = ro_multidx.get_level_values('side').to_list()
mn_labels = ro_multidx.get_level_values('muscle').to_list()
mn_rank = ro_multidx.get_level_values('rank').to_list()
mn_ids = ro_multidx.get_level_values('segID').to_list()
idx = [str(i) for i in range(len(mn_ids))]
sid = [str(i) for i in mn_ids]
mn_rank = [str(i) for i in mn_rank]
lbls = [h + '_' + i + '_' + j + '_' + k + '_' + l for h, i, j, k, l in zip(mn_side,mn_labels, mn_rank, sid, idx,)]

fig = plt.figure(1, figsize = [20,18])
ax = sea.heatmap(sim_mat, xticklabels=mn_ids, yticklabels=lbls)
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

In [ ]:
def printMNstuff(i,mltidx):
    idx = mltidx[i]
    print(idx)
    segids = t1_mns_df.pt_root_id.to_list()
    df_i = segids.index(idx[-1])
    pnt = t1_mns_df.pt_position[df_i]
    print(pnt)
    
def print_pre_stuff(i,idxlist):
    idx = idxlist[i]
    print(idx)

In [ ]:
# Trochanter promotors
printMNstuff(0,ro_multidx) # input to main tibia flexors
printMNstuff(1,ro_multidx) # input to main tibia flexors 648518346490952225
printMNstuff(2,ro_multidx) # input to main tibia flexors 648518346494806962
printMNstuff(3,ro_multidx) # input to main tibia flexors 648518346495328079
# 648518346486120840 648518346504764531 648518346500284085 648518346475233889

printMNstuff(45,ro_multidx) # input to main tibia flexors 648518346483181988
#648518346483181988

print('')
print('flexors')
# Trochanter flexors
printMNstuff(4,ro_multidx) # input to main tibia flexors
printMNstuff(5,ro_multidx) # input to main tibia flexors 648518346490952225
printMNstuff(6,ro_multidx) # input to main tibia flexors 648518346494806962
printMNstuff(7,ro_multidx) # input to main tibia flexors 648518346495328079
printMNstuff(8,ro_multidx) # input to main tibia flexors 648518346483087268
printMNstuff(9,ro_multidx) # input to main tibia flexors 648518346475428480
# 648518346478862543 648518346507176223 648518346483112382 648518346489322585 648518346499251291 648518346517668968

In [ ]:
# Note, these change from instance to instance for some reason
# Miller 32 and anterior motor neurons: https://neuromancer-seung-import.appspot.com/?json_url=https://global.daf-apis.com/nglstate/api/v1/4566692181049344
printMNstuff(20,ro_multidx) # input to main tibia flexors 648518346486098834 648518346490952225 648518346494806962 648518346495328079 648518346483087268 648518346475428480
printMNstuff(21,ro_multidx) # input to main tibia flexors 648518346490952225
printMNstuff(22,ro_multidx) # input to main tibia flexors 648518346494806962
printMNstuff(23,ro_multidx) # input to main tibia flexors 648518346495328079
printMNstuff(24,ro_multidx) # input to main tibia flexors 648518346483087268
printMNstuff(25,ro_multidx) # input to main tibia flexors 648518346475428480
# 648518346486098834 648518346490952225 648518346494806962 648518346495328079 648518346483087268 648518346475428480



In [ ]:
# Trochanteral extension (depression): https://neuromancer-seung-import.appspot.com/?json_url=https://global.daf-apis.com/nglstate/api/v1/5678571838242816
printMNstuff(51,ro_multidx) # input to main tibia flexors 648518346486706585
printMNstuff(52,ro_multidx) # input to main tibia flexors 648518346486706585
printMNstuff(53,ro_multidx) # input to main tibia flexors 648518346486706585
printMNstuff(54,ro_multidx) # input to main tibia flexors 648518346486706585
printMNstuff(55,ro_multidx) # input to main tibia flexors 648518346486706585

In [ ]:
# weird trochanter extensor
printMNstuff(43,ro_multidx) # 
printMNstuff(44,ro_multidx) # 
# 648518346493661944 648518346489611133

In [ ]:
# stray small ltms
printMNstuff(47,ro_multidx) # 
# 648518346497452278 648518346489317209

In [ ]:
# stray small ltms
printMNstuff(0,ro_multidx) # 
# 648518346497452278 648518346489317209

In [ ]:
# Femur ltms
printMNstuff(65,ro_multidx) # 
printMNstuff(66,ro_multidx) # 
# 648518346493757553 648518346507111240

# Connectome of preMNs
Premotor neurons can: 
 - have a soma
     - local
     - intersegmental
          - outputs to T1
          - mixed inputs/outputs in T1
              - more inputs in T1
              - more outpus in T1
     - ascending
 - have no soma
     - descending
     - sensory
     - fragments

## With somas

In [ ]:
ro_multidx = fullfig2order_wsoma_df.columns
sim_mat = cosine_similarity(fullfig2order_wsoma_df.to_numpy())

# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None).fit(sim_mat)

# plot the top three levels of the dendrogram
clustered_order = utils.plot_dendrogram(model, truncate_mode="level", p=6000) # p truncate mode
clustered_order = np.array(clustered_order).astype(int) # convert strins into integers
# print(clustered_order)
reordered_pre_to_mn_df = fullfig2order_wsoma_df.iloc[clustered_order,:]

### Plot cosine similarity

In [ ]:
sim_mat = cosine_similarity(reordered_pre_to_mn_df.to_numpy())

# visualize matrix
mn_labels = ro_multidx.get_level_values('muscle').to_list()
mn_rank = ro_multidx.get_level_values('rank').to_list()
mn_ids = ro_multidx.get_level_values('segID').to_list()
idx = [str(i) for i in range(len(mn_ids))]
sid = [str(i) for i in mn_ids]
mn_rank = [str(i) for i in mn_rank]
lbls = [i + '_' + j + '_' + k + '_' + l for i, j, k, l in zip(mn_labels, mn_rank, sid, idx,)]

fig = plt.figure(1, figsize = [20,18])
ax = sea.heatmap(sim_mat) #, xticklabels=mn_ids, yticklabels=lbls)
# ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
#ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Premotor Neurons', fontsize =18)
plt.ylabel('Premotor Neurons', fontsize =18)
# plt.yticks(fontsize = 16)
# plt.xticks(fontsize = 16)
plt.show()

### Full connectome onto right and left sides. TODO: order by intersegmental vs local

In [ ]:
reorder_mn_mat = np.log10(reordered_pre_to_mn_df.to_numpy().transpose()+1)
fig = plt.figure(1, figsize = [25,15])
ax = sea.heatmap(reorder_mn_mat, yticklabels=lbls) #, cmap=parula_map
cbar = ax.collections[0].colorbar
# here set the labelsize by 20
# cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'log 10 # of synapses', size=24)
plt.xlabel('Motor Neuron SegIDS', fontsize =18)
plt.ylabel('Hair Plate Identity', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

# Connectome of no soma neurons

In [ ]:
reorder_idx_bysegfcn = utils.get_segment_fcn_order()
# pre_wsoma_to_mn_df = pre_to_mn_df.loc[(All, True), :]
fullfig2order_df = pre_to_mn_df.iloc[:,reorder_idx_bysegfcn]
fullfig2order_df.shape
# sim_mat = fullfig2order_df.to_numpy().transpose()

In [ ]:
#reordered_mn_df = pre_to_mn_df.iloc[:,clustered_order]
sim_mat = cosine_similarity(fullfig2order_df.to_numpy())
ro_multidx = fullfig2order_df.columns

# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None).fit(sim_mat)

# plot the top three levels of the dendrogram
clustered_order = utils.plot_dendrogram(model, truncate_mode="level", p=6000) # p truncate mode
clustered_order = np.array(clustered_order).astype(int) # convert strins into integers
print(clustered_order)
reordered_pre_to_mn_df = fullfig2order_df.iloc[clustered_order,:]
reordered_pre_to_mn_df.shape

## Plot cosine similarity

In [ ]:
sim_mat = cosine_similarity(reordered_pre_to_mn_df.to_numpy())

# visualize matrix
mn_labels = ro_multidx.get_level_values('muscle').to_list()
mn_rank = ro_multidx.get_level_values('rank').to_list()
mn_ids = ro_multidx.get_level_values('segID').to_list()
idx = [str(i) for i in range(len(mn_ids))]
sid = [str(i) for i in mn_ids]
mn_rank = [str(i) for i in mn_rank]
lbls = [i + '_' + j + '_' + k + '_' + l for i, j, k, l in zip(mn_labels, mn_rank, sid, idx,)]

fig = plt.figure(1, figsize = [20,18])
ax = sea.heatmap(sim_mat) #, xticklabels=mn_ids, yticklabels=lbls)
# ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
#ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Premotor Neurons', fontsize =18)
plt.ylabel('Premotor Neurons', fontsize =18)
# plt.yticks(fontsize = 16)
# plt.xticks(fontsize = 16)
plt.show()

## Full connectome onto right and left sides

In [ ]:
reorder_mn_mat = np.log10(reordered_pre_to_mn_df.to_numpy().transpose()+1)
fig = plt.figure(1, figsize = [25,15])
ax = sea.heatmap(reorder_mn_mat, yticklabels=lbls) #, cmap=parula_map
cbar = ax.collections[0].colorbar
# here set the labelsize by 20
# cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'log 10 # of synapses', size=24)
plt.xlabel('Motor Neuron SegIDS', fontsize =18)
plt.ylabel('Hair Plate Identity', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

## Find descending neurons
Descending neurons are likely those with the most input and no somas.
Try to make a table to not duplicate the effort.

In [ ]:
## 

In [ ]:
pre_topnosoma_df = pre_to_mn_df.loc[(All, False), :]

# Find the total number of inputs
s = pre_topnosoma_df.sum(axis=1)
s
pre_topnosoma_df.loc[:,'total'] = s
pre_topnosoma_df

In [ ]:
pre_topnosoma_sorted_df= pre_topnosoma_df.sort_values(by='total',ascending=False)
# pre_topnosoma_sorted_df.loc[pre_topnosoma_df.total>100,:] # This list is completed 4/25/22
pre_topnosoma_sorted_df.loc[(pre_topnosoma_df.total>50) & (pre_topnosoma_df.total<=100),:] # This list is completed 4/25/22

# order = totinputs.
today = date.today()
d1 = today.strftime("%Y%m%d")
#pre_topnosoma_sorted_df.loc[(pre_topnosoma_df.total>50) & (pre_topnosoma_df.total<=100),:].to_csv('./saved_dfs/nosoma_partners_' + d1 + '.csv')
pre_topnosoma_sorted_df.loc[(pre_topnosoma_df.total>50)].to_csv('./saved_dfs/nosoma_partners_' + d1 + '.csv')
print(pre_topnosoma_sorted_df.loc[(pre_topnosoma_df.total>50) & (pre_topnosoma_df.total<=100),:].shape)


## Find intersegmental neurons: have a soma, then distribution of synapses

In [ ]:
# T1 bounding box, from dacks lab
x = [42542, 152542]
y = [392291.4,532291.4]
z = [37065,162065]

# Produce lists of neurons to edit

In [ ]:
# Output the entire list. 
# Have three columns: 
# 1)the seg id of the premotor neuron, 
# 2) the segid of the motor neuron receiving the most input, 
# 3) and the number of synapses
All = slice(None)

mn_segids = reordered_pre_to_mn_df.columns.get_level_values('segID').to_list()
mn_labels = reordered_pre_to_mn_df.columns.get_level_values('muscle').to_list()
mn_rank = reordered_pre_to_mn_df.columns.get_level_values('rank').to_list()
mn_rank = [str(i) for i in mn_rank]
mn_lbls = [i + '_' + j for i, j in zip(mn_labels, mn_rank)]

# create a square dataframe of 1s
dfdata = np.zeros([reordered_pre_to_mn_df.shape[0],3],dtype=np.uint64)
temp_df = pd.DataFrame(dfdata,index=reordered_pre_to_mn_df.index, columns=['top_MN_partner','label','synapse_cnt'])
                  
for segid1, row in reordered_pre_to_mn_df.iterrows():
    rownp = row.to_numpy()
    idx = rownp.argmax()
    temp_df.loc[segid1,'top_MN_partner'] = mn_segids[idx]
    temp_df.loc[segid1,'label'] = mn_lbls[idx]
    temp_df.loc[segid1,'synapse_cnt'] = rownp[idx]
    

today = date.today()
d1 = today.strftime("%Y%m%d")
temp_df.to_csv('./saved_dfs/sortedPresyapticNeuronsAndMajorPartner_' + d1 + '.csv')
print(temp_df.shape)



In [ ]:
# Output sources that provide input to a single motor neuron
# Have three columns: 
# 1)the seg id of the premotor neuron, 
# 2) the segid of the motor neuron receiving the most input, 
# 3) and the number of synapses
All = slice(None)

mn_segids = reordered_pre_to_mn_df.columns.get_level_values('segID').to_list()
mn_labels = reordered_pre_to_mn_df.columns.get_level_values('muscle').to_list()
mn_rank = reordered_pre_to_mn_df.columns.get_level_values('rank').to_list()
mn_rank = [str(i) for i in mn_rank]
mn_lbls = [i + '_' + j for i, j in zip(mn_labels, mn_rank)]

# create a square dataframe of 1s
dfdata = np.zeros([reordered_pre_to_mn_df.shape[0],3],dtype=np.uint64)
singletarget_df = pd.DataFrame(dfdata,index=reordered_pre_to_mn_df.index, columns=['top_MN_partner','label','synapse_cnt'])
singletarget_nosoma_df = pd.DataFrame(dfdata,index=reordered_pre_to_mn_df.index, columns=['top_MN_partner','label','synapse_cnt'])
cnt = 0
nosomacnt = 0

for segid1, row in reordered_pre_to_mn_df.iterrows():
    rownp = row.to_numpy()
    if np.sum(rownp>0) > 1:
        cnt = cnt+1;
        continue
    elif segid1 not in soma_table.pt_root_id.to_list():
        nosomacnt = nosomacnt+1
        idx = rownp.argmax()
        singletarget_nosoma_df.loc[segid1,'top_MN_partner'] = mn_segids[idx]
        singletarget_nosoma_df.loc[segid1,'label'] = mn_lbls[idx]
        singletarget_nosoma_df.loc[segid1,'synapse_cnt'] = rownp[idx]
        
    singletarget_df.loc[segid1,'top_MN_partner'] = mn_segids[idx]
    singletarget_df.loc[segid1,'label'] = mn_lbls[idx]
    singletarget_df.loc[segid1,'synapse_cnt'] = rownp[idx]
    
today = date.today()
d1 = today.strftime("%Y%m%d")

singletarget_nosoma_df = singletarget_nosoma_df.loc[singletarget_nosoma_df['synapse_cnt']>0,All]
singletarget_df = singletarget_df.loc[singletarget_df['synapse_cnt']>0,All]

singletarget_nosoma_df.to_csv('./saved_dfs/sortedmninputs_singleinput_nosoma_' + d1 +'.csv')
singletarget_df.to_csv('./saved_dfs/sortedmninputs_singleinput_' + d1 +'.csv')

singletarget_df.head()
print(singletarget_nosoma_df.shape)
print(singletarget_df.shape)

In [ ]:
# Output list of inputs to FETi and SETi
Tidf = reordered_pre_to_mn_df.loc[:,(All,All,All,All,All,['feti','seti'],All)]
# reordered_pre_to_mn_df.columns
maxidx = np.argsort(np.max(Tidf.to_numpy(),axis=1))
print(maxidx.shape)
Tidf_ordrd = Tidf.iloc[np.flipud(maxidx),All]
today = date.today()
d1 = today.strftime("%Y%m%d")
Tidf_ordrd.to_csv('./saved_dfs/sortedmninputs_feti_seti' + d1 +'.csv')
#Tidf_ordrd_no0 = Tidf_ordrd.loc[Tidf_ordrd['feti']>0 or Tidf_ordrd['seti'],All]
Tidf_ordrd.head()
# dfdata = np.zeros([reordered_pre_to_mn_df.shape[0],3],dtype=np.uint64)
# singletarget_df = pd.DataFrame(dfdata,index=reordered_pre_to_mn_df.index, columns=['top_MN_partner','label','synapse_cnt'])
# singletarget_nosoma_df = pd.DataFrame(dfdata,index=reordered_pre_to_mn_df.index, columns=['top_MN_partner','label','synapse_cnt'])
# cnt = 0
# nosomacnt = 0

# for segid1, row in reordered_pre_to_mn_df.iterrows():
#     rownp = row.to_numpy()
#     if np.sum(rownp>0) > 1:
#         cnt = cnt+1;
#         continue
#     elif segid1 not in soma_table.pt_root_id.to_list():
#         nosomacnt = nosomacnt+1
#         idx = rownp.argmax()
#         singletarget_nosoma_df.loc[segid1,'top_MN_partner'] = mn_segids[idx]
#         singletarget_nosoma_df.loc[segid1,'label'] = mn_lbls[idx]
#         singletarget_nosoma_df.loc[segid1,'synapse_cnt'] = rownp[idx]
        
#     singletarget_df.loc[segid1,'top_MN_partner'] = mn_segids[idx]
#     singletarget_df.loc[segid1,'label'] = mn_lbls[idx]
#     singletarget_df.loc[segid1,'synapse_cnt'] = rownp[idx]

# TODO:
- Analyse the Left R neurons
- What neurons are contributing to the similarity across segments?